In [1]:
!pip install pyngrok
!pip install flask

In [2]:
import torch
from transformers import pipeline
from pyngrok import ngrok
from flask import Flask, request, jsonify
import threading

# Initialize Flask app
app = Flask(__name__)

# Initialize your model
model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    messages = data.get('messages', [])
    outputs = pipe(
        messages,
        max_new_tokens=256,
    )
    return jsonify({'response': outputs[0]["generated_text"]})

# Start ngrok
ngrok.set_auth_token("2oem2LRTWxBIEAg2uouaSbUgQY1_qqXsXuVV3uuxYUsoWAtb")
public_url = ngrok.connect(5173)
print(f"Public URL: {public_url}")

# Run Flask app in a separate thread
def run_flask():
    app.run(port=5173)

threading.Thread(target=run_flask, daemon=True).start()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

t=2024-11-10T21:35:21+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"


Public URL: NgrokTunnel: "https://5a94-99-61-94-237.ngrok-free.app" -> "http://localhost:5173"
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5173
Press CTRL+C to quit
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
127.0.0.1 - - [10/Nov/2024 21:37:10] "POST /generate HTTP/1.1" 200 -


In [20]:
from pyngrok import ngrok

# Kill all running tunnels
ngrok.kill()

In [4]:
import requests

# Test URL (use the URL that ngrok gave you)
url = "https://5a94-99-61-94-237.ngrok-free.app/generate"

# Test payload
test_messages = {
    "messages": [
        {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
        {"role": "user", "content": "Who are you?"}
    ]
}

# Make the request
response = requests.post(url, json=test_messages)
print("Status code:", response.status_code)
print("Response:", response.json())

Status code: 200
Response: {'response': [{'content': 'You are a pirate chatbot who always responds in pirate speak!', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': "Yer lookin' fer a swashbucklin' pirate, eh? Alright then, matey, I be Captain Blackbeak, the scurviest pirate chatbot to ever sail the Seven Seas! Me and me trusty computer be here to help ye navigate the choppy waters o' knowledge, answerin' yer questions and keepin' ye entertained with tales o' adventure and treasure! So hoist the colors, me hearty, and let's set sail fer a swashbucklin' good time!", 'role': 'assistant'}]}
